##  Extreme Gradient Boosting Outlier Detection


Take the advantage of both supervised and unsupervised learning: utiliser la capacité des méthodes de classification non supervisée pour détercter les outlier et la précision des techniques de classfications supervisée pour la prévision avec précision. 

XGBOD has three steps:

1- unsupervised learning to create new features, called Transformed Outlier Scores (TOS). includes KNN, AvgKNN, LOF, iForest, HBOS, and OCSVM

2- concatenates the new features with the original features, then applies Pearson’s correlation coefficients to keep the useful features

3- trains an XGBoost classifier

In [ ]:
from pyod.models.xgbod import XGBOD

In [ ]:
# Diviser les données en ensembles d'entraînement et de test pour X et Y
X = data_no_nan_for_XGBOD.drop('Binar OP130_Resultat_Global_v', axis =1)  # Caractéristiques
Y = data_no_nan_for_XGBOD['Binar OP130_Resultat_Global_v']  # Variable cible

X_train_XGBOD, X_test_XGBOD, Y_train_XGBOD, Y_test_XGBOD = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
xgbod = XGBOD(n_components=3,random_state=100) 
xgbod.fit(X_train_XGBOD,Y_train_XGBOD)

XGBOD(base_score=0.5, booster='gbtree', colsample_bylevel=1,
   colsample_bytree=1,
   estimator_list=[KNN(algorithm='auto', contamination=0.1, leaf_size=30, method='largest',
  metric='minkowski', metric_params=None, n_jobs=1, n_neighbors=1, p=2,
  radius=1.0), LOF(algorithm='auto', contamination=0.1, leaf_size=30, metric='minkowski',
  metric_params=None, n_jobs=1, n_neighbors=1, no..._features=1.0,
    max_samples='auto', n_estimators=200, n_jobs=1, random_state=100,
    verbose=0)],
   gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
   min_child_weight=1, n_estimators=100, n_jobs=1, nthread=None,
   objective='binary:logistic', random_state=100, reg_alpha=0,
   reg_lambda=1, scale_pos_weight=1, silent=True,
   standardization_flag_list=[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, False, False, False, False, False, False, True, True, True, True, True, True, True, True, True, True, True, False, False, False, False,

In [ ]:
# get the prediction labels and outlier scores of the training data
y_train_pred = xgbod.labels_  # binary labels (0: inliers, 1: outliers)
y_train_scores = xgbod.decision_scores_  # raw outlier scores
y_train_scores = xgbod.decision_function(X_train_XGBOD)

# get the prediction on the test data
y_test_pred = xgbod.predict(X_test_XGBOD)  # outlier labels (0 or 1)
y_test_scores = xgbod.decision_function(X_test_XGBOD)  # outlier scores

def count_stat(vector):
    # Because it is '0' and '1', we can run a count statistic. 
    unique, counts = np.unique(vector, return_counts=True)
    return dict(zip(unique, counts))

print("The training data:", count_stat(y_train_pred))
print("The test data:", count_stat(y_test_pred))

The training data: {0: 12710}
The test data: {0: 3178}


In [ ]:
Actual_pred = pd.DataFrame({'Actual': Y_test_XGBOD, 'Pred': y_test_pred})
pd.crosstab(Actual_pred['Actual'],Actual_pred['Pred'])

Pred,0
Actual,
0,3150
1,28


In [ ]:
# Let's see how many '0's and '1's.
df_train = pd.DataFrame(X_train_XGBOD)
df_columns = df_train.columns
df_train['pred'] = y_train_pred
df_train['Group'] = np.where(df_train['pred']==1, 'Outlier','Normal')

# Now let's show the summary statistics:
cnt = df_train.groupby('Group')['pred'].count().reset_index().rename(columns={'pred':'Count'})
cnt['Count %'] = (cnt['Count'] / cnt['Count'].sum()) * 100 # The count and count %
stat = df_train.groupby('Group').mean().reset_index() # The avg.
cnt.merge(stat, left_on='Group',right_on='Group') # Put the count and the avg. together


,Group,Count,Count %,OP070_V_1_angle_value,OP090_SnapRingPeakForce_value,OP070_V_2_angle_value,OP120_Rodage_I_mesure_value,OP090_SnapRingFinalStroke_value,OP110_Vissage_M8_torque_value,OP100_Capuchon_insertion_mesure,OP120_Rodage_U_mesure_value,OP070_V_1_torque_value,OP090_StartLinePeakForce_value,OP110_Vissage_M8_angle_value,OP090_SnapRingMidPointForce_val,OP070_V_2_torque_value,pred
0,Normal,12710,100.0,158.386703,150.592286,157.723619,113.08681,11.994315,12.264671,0.388153,11.970401,6.477945,22.651002,17.527537,95.790449,6.4821,0.0
